In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Load dữ liệu Iris
iris = load_iris()
X = iris.data
y = iris.target

In [9]:
print("Trần Phương Anh - 6351071002")
print(f"Số lượng mẫu: {X.shape[0]}")
print(f"Số lượng features: {X.shape[1]}")
print(f"Số lớp: {len(np.unique(y))}")

Trần Phương Anh - 6351071002
Số lượng mẫu: 150
Số lượng features: 4
Số lớp: 3


# Câu c


In [6]:
def create_model(input_dim=4, hidden_units=[16, 8], num_classes=3):
    """
    Tạo mô hình Neural Network cho phân loại Iris

    Tham số:
    - input_dim: Số lượng features đầu vào (4 cho Iris)
    - hidden_units: List số neurons cho các hidden layers
    - num_classes: Số lớp đầu ra (3 cho Iris)
    """
    model = keras.Sequential([
        # Input layer
        keras.layers.Input(shape=(input_dim,)),

        # Hidden layer 1
        keras.layers.Dense(hidden_units[0], activation='relu', name='hidden_1'),
        keras.layers.Dropout(0.2),

        # Hidden layer 2
        keras.layers.Dense(hidden_units[1], activation='relu', name='hidden_2'),
        keras.layers.Dropout(0.2),

        # Output layer
        keras.layers.Dense(num_classes, activation='softmax', name='output')
    ])

    # Biên dịch mô hình
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


# Câu d

In [10]:
k = 10
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

# Lưu kết quả của từng fold
fold_results = []
fold_number = 1

print("Trần Phương Anh - 6351071002")
print("\nBẮT ĐẦU HUẤN LUYỆN VỚI 10-FOLD CROSS VALIDATION")
print("=" * 60)

for train_idx, test_idx in kfold.split(X):
    print(f"\n{'='*60}")
    print(f"FOLD {fold_number}/{k}")
    print(f"{'='*60}")

    # Chia dữ liệu
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Tạo mô hình mới cho mỗi fold
    model = create_model(input_dim=4, hidden_units=[16, 8], num_classes=3)

    # Huấn luyện mô hình
    print(f"\nĐang huấn luyện Fold {fold_number}...")
    history = model.fit(
        X_train_scaled, y_train,
        epochs=100,
        batch_size=8,
        verbose=0,  # Tắt output chi tiết
        validation_split=0.2
    )

    # Dự đoán trên tập test
    y_pred_probs = model.predict(X_test_scaled, verbose=0)
    y_pred = np.argmax(y_pred_probs, axis=1)

    # Tính các metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # In kết quả
    print(f"\nKẾT QUẢ FOLD {fold_number}:")
    print(f"  Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1-Score:  {f1:.4f}")

    # Lưu kết quả
    fold_results.append({
        'fold': fold_number,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    })

    # Lưu mô hình
    model_filename = f'iris_model_fold_{fold_number}.h5'
    model.save(model_filename)
    print(f"\n✓ Đã lưu mô hình: {model_filename}")

    fold_number += 1

# Tính toán và in kết quả trung bình
print("\n" + "=" * 60)
print("KẾT QUẢ TỔNG HỢP TẤT CẢ FOLDS")
print("=" * 60)

avg_accuracy = np.mean([r['accuracy'] for r in fold_results])
avg_precision = np.mean([r['precision'] for r in fold_results])
avg_recall = np.mean([r['recall'] for r in fold_results])
avg_f1 = np.mean([r['f1_score'] for r in fold_results])

std_accuracy = np.std([r['accuracy'] for r in fold_results])
std_precision = np.std([r['precision'] for r in fold_results])
std_recall = np.std([r['recall'] for r in fold_results])
std_f1 = np.std([r['f1_score'] for r in fold_results])

print(f"\nKẾT QUẢ TRUNG BÌNH:")
print(f"  Accuracy:  {avg_accuracy:.4f} ± {std_accuracy:.4f}")
print(f"  Precision: {avg_precision:.4f} ± {std_precision:.4f}")
print(f"  Recall:    {avg_recall:.4f} ± {std_recall:.4f}")
print(f"  F1-Score:  {avg_f1:.4f} ± {std_f1:.4f}")

# Bảng kết quả chi tiết
print(f"\n{'='*60}")
print("BẢNG KẾT QUẢ CHI TIẾT TỪNG FOLD")
print(f"{'='*60}")
print(f"{'Fold':<6} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12}")
print("-" * 60)
for result in fold_results:
    print(f"{result['fold']:<6} "
          f"{result['accuracy']:<12.4f} "
          f"{result['precision']:<12.4f} "
          f"{result['recall']:<12.4f} "
          f"{result['f1_score']:<12.4f}")
print("=" * 60)

# Tìm fold tốt nhất
best_fold = max(fold_results, key=lambda x: x['accuracy'])
print(f"\n✓ FOLD TỐT NHẤT: Fold {best_fold['fold']} với Accuracy = {best_fold['accuracy']:.4f}")
print(f"✓ ĐÃ LƯU {k} MÔ HÌNH: iris_model_fold_1.h5 đến iris_model_fold_{k}.h5")

Trần Phương Anh - 6351071002

BẮT ĐẦU HUẤN LUYỆN VỚI 10-FOLD CROSS VALIDATION

FOLD 1/10

Đang huấn luyện Fold 1...



KẾT QUẢ FOLD 1:
  Accuracy:  1.0000 (100.00%)
  Precision: 1.0000
  Recall:    1.0000
  F1-Score:  1.0000

✓ Đã lưu mô hình: iris_model_fold_1.h5

FOLD 2/10

Đang huấn luyện Fold 2...



KẾT QUẢ FOLD 2:
  Accuracy:  0.9333 (93.33%)
  Precision: 0.9500
  Recall:    0.9333
  F1-Score:  0.9359

✓ Đã lưu mô hình: iris_model_fold_2.h5

FOLD 3/10

Đang huấn luyện Fold 3...



KẾT QUẢ FOLD 3:
  Accuracy:  1.0000 (100.00%)
  Precision: 1.0000
  Recall:    1.0000
  F1-Score:  1.0000

✓ Đã lưu mô hình: iris_model_fold_3.h5

FOLD 4/10

Đang huấn luyện Fold 4...



KẾT QUẢ FOLD 4:
  Accuracy:  0.9333 (93.33%)
  Precision: 0.9429
  Recall:    0.9333
  F1-Score:  0.9322

✓ Đã lưu mô hình: iris_model_fold_4.h5

FOLD 5/10

Đang huấn luyện Fold 5...


...


KẾT QUẢ FOLD 6:
  Accuracy:  0.8667 (86.67%)
  Precision: 0.9200
  Recall:    0.8667
  F1-Score:  0.8700

✓ Đã lưu mô hình: iris_model_fold_6.h5

FOLD 7/10

Đang huấn luyện Fold 7...



KẾT QUẢ FOLD 7:
  Accuracy:  0.8667 (86.67%)
  Precision: 0.9111
  Recall:    0.8667
  F1-Score:  0.8667

✓ Đã lưu mô hình: iris_model_fold_7.h5

FOLD 8/10

Đang huấn luyện Fold 8...



KẾT QUẢ FOLD 8:
  Accuracy:  1.0000 (100.00%)
  Precision: 1.0000
  Recall:    1.0000
  F1-Score:  1.0000

✓ Đã lưu mô hình: iris_model_fold_8.h5

FOLD 9/10

Đang huấn luyện Fold 9...



KẾT QUẢ FOLD 9:
  Accuracy:  0.9333 (93.33%)
  Precision: 0.9444
  Recall:    0.9333
  F1-Score:  0.9327

✓ Đã lưu mô hình: iris_model_fold_9.h5

FOLD 10/10

Đang huấn luyện Fold 10...



KẾT QUẢ FOLD 10:
  Accuracy:  0.9333 (93.33%)
  Precision: 0.9429
  Recall:    0.9333
  F1-Score:  0.9333

✓ Đã lưu mô hình: iris_model_fold_10.h5

KẾT QUẢ TỔNG HỢP TẤT CẢ FOLDS

KẾT QUẢ TRUNG BÌNH:
  Accuracy:  0.9467 ± 0.0499
  Precision: 0.9611 ± 0.0336
  Recall:    0.9467 ± 0.0499
  F1-Score:  0.9471 ± 0.0493

BẢNG KẾT QUẢ CHI TIẾT TỪNG FOLD
Fold   Accuracy     Precision    Recall       F1-Score    
------------------------------------------------------------
1      1.0000       1.0000       1.0000       1.0000      
2      0.9333       0.9500       0.9333       0.9359      
3      1.0000       1.0000       1.0000       1.0000      
4      0.9333       0.9429       0.9333       0.9322      
5      1.0000       1.0000       1.0000       1.0000      
6      0.8667       0.9200       0.8667       0.8700      
7      0.8667       0.9111       0.8667       0.8667      
8      1.0000       1.0000       1.0000       1.0000      
9      0.9333       0.9444       0.9333       0.9327      
